In [4]:
import numpy as np
from scipy.optimize import minimize

class SupplyChainPH:
    def __init__(self, num_facilities, num_customers, num_scenarios, fixed_costs, transport_costs, capacities, demands, probabilities, rho=1.0):
        self.num_facilities = num_facilities
        self.num_customers = num_customers
        self.num_scenarios = num_scenarios
        self.fixed_costs = fixed_costs
        self.transport_costs = transport_costs
        self.capacities = capacities
        self.demands = demands
        self.probabilities = probabilities
        self.rho = rho

        # Initialize variables
        self.x = np.random.rand(num_scenarios, num_facilities, num_customers)
        self.w = np.zeros((num_scenarios, num_facilities, num_customers))
        self.z = np.mean(self.x, axis=0)

    def scenario_subproblem(self, scenario):
        def objective(x):
            x_reshaped = x.reshape(self.num_facilities, self.num_customers)
            fixed_cost = np.sum(self.fixed_costs * (np.sum(x_reshaped, axis=1) > 0))
            transport_cost = np.sum(self.transport_costs * x_reshaped)
            augmented_lagrangian = np.sum(self.w[scenario] * (x_reshaped - self.z)) + \
                                   (self.rho/2) * np.sum((x_reshaped - self.z)**2)
            return fixed_cost + transport_cost + augmented_lagrangian

        def constraints(x):
            x_reshaped = x.reshape(self.num_facilities, self.num_customers)
            capacity_constraint = self.capacities - np.sum(x_reshaped, axis=1)
            demand_constraint = np.sum(x_reshaped, axis=0) - self.demands[scenario]
            return np.concatenate([capacity_constraint, demand_constraint])

        x0 = self.x[scenario].flatten()
        cons = {'type': 'ineq', 'fun': constraints}
        bounds = [(0, None) for _ in range(self.num_facilities * self.num_customers)]

        result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)
        return result.x.reshape(self.num_facilities, self.num_customers)

    def calculate_objective(self, x):
        fixed_cost = np.sum(self.fixed_costs * (np.sum(x, axis=1) > 0))
        transport_cost = np.sum(self.transport_costs * x)
        return fixed_cost + transport_cost

    def solve(self, max_iterations=1000, tolerance=1e-4):
        for iteration in range(max_iterations):
            # Solve scenario subproblems
            for s in range(self.num_scenarios):
                self.x[s] = self.scenario_subproblem(s)

            # Compute new average solution
            z_new = np.average(self.x, axis=0, weights=self.probabilities)

            # Calculate and print objective value
            obj_value = np.sum([self.probabilities[s] * self.calculate_objective(self.x[s]) for s in range(self.num_scenarios)])
            print(f"Iteration {iteration+1}, Objective Value: {obj_value:.2f}")

            # Update price variables
            for s in range(self.num_scenarios):
                self.w[s] += self.rho * (self.x[s] - z_new)

            # Check convergence
            if np.max(np.abs(z_new - self.z)) < tolerance:
                break

            self.z = z_new

        final_obj_value = self.calculate_objective(self.z)
        print(f"\nFinal Objective Value: {final_obj_value:.2f}")
        return self.z

# Example usage
np.random.seed(42)  # for reproducibility
num_facilities = 3
num_customers = 5
num_scenarios = 3

fixed_costs = np.array([100000, 120000, 90000])
transport_costs = np.random.rand(num_facilities, num_customers) * 100
capacities = np.array([5000, 6000, 4500])
demands = np.random.randint(500, 1500, size=(num_scenarios, num_customers))
probabilities = np.array([0.3, 0.5, 0.2])

ph_solver = SupplyChainPH(num_facilities, num_customers, num_scenarios, 
                          fixed_costs, transport_costs, capacities, demands, probabilities)

solution = ph_solver.solve()

print("\nOptimal facility allocations:")
print(solution)
print("\nFacilities to open:")
print(np.where(np.sum(solution, axis=1) > 0)[0])

Iteration 1, Objective Value: 555423.39
Iteration 2, Objective Value: 572566.87
Iteration 3, Objective Value: 574556.58
Iteration 4, Objective Value: 572047.39
Iteration 5, Objective Value: 565863.42
Iteration 6, Objective Value: 556861.62
Iteration 7, Objective Value: 546683.37
Iteration 8, Objective Value: 536574.18
Iteration 9, Objective Value: 526994.69
Iteration 10, Objective Value: 517734.77
Iteration 11, Objective Value: 508461.08
Iteration 12, Objective Value: 501169.43
Iteration 13, Objective Value: 494223.16
Iteration 14, Objective Value: 488554.09
Iteration 15, Objective Value: 487003.05
Iteration 16, Objective Value: 485576.55
Iteration 17, Objective Value: 484088.33
Iteration 18, Objective Value: 482628.52
Iteration 19, Objective Value: 481240.32
Iteration 20, Objective Value: 479940.00
Iteration 21, Objective Value: 478727.02
Iteration 22, Objective Value: 477593.31
Iteration 23, Objective Value: 476507.16
Iteration 24, Objective Value: 475430.94
Iteration 25, Objective V